(1) Importing dependency

In [1]:
import keras
from keras.models import Sequential, load_model
from keras.layers import Dense, Activation, Dropout, Flatten,\
                         Conv1D, MaxPooling1D
from keras.layers.normalization import BatchNormalization
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.utils import to_categorical

import sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.utils import shuffle

import pandas as pd
import librosa
from tqdm import tqdm_notebook as tqdm
import os

import numpy as np
np.random.seed(13)
import random
random.seed(13)

import matplotlib.pyplot as plt
%matplotlib inline

Using TensorFlow backend.


(2) Get Data

In [2]:
base_dir = 'data/'
data_dir = base_dir+'data_speech_commands_v0.02/'
 
train_txt = base_dir+'train_16words.txt'
val_txt = base_dir+'validation_16words.txt'
test_txt = base_dir+'test_16words.txt'

In [3]:
class_list = ['zero', 'one', 'two', 'three', 'four', 
              'five', 'six', 'seven', 'eight', 'nine', 
              'bed', 'bird', 'tree', 'cat', 'house',
              'dog']

class_dict = {_class:i for i, _class in enumerate(class_list)}

In [4]:
def load_wavs(filenames):
    first_wav, _ = librosa.load(filenames[0])
    wav_shape = first_wav.shape[0]
    x_data = [librosa.util.fix_length(librosa.load(filename)[0], wav_shape) 
              for filename in tqdm(filenames)]
    return np.asarray(x_data)
    
### If you have lack of memory, Use this
#     wav, _ = librosa.load(filenames[0])
#     wavs = np.zeros( (len(filenames), wav.shape[0]) )
#     for i, filename in enumerate(filenames):
#         wavs[i] = librosa.load(filename)[0][:]
#     return wavs

def make_x_data(filenames):
    x_data = load_wavs(filenames)
    x_data = np.reshape(x_data, (*x_data.shape, 1))
    return x_data
    
def extract_class_from_filename(filename):
    dirname = os.path.dirname(filename)
    class_name = dirname.split('/')[-1]
    return class_name

def make_y_data(filenames, y_dict):
    return np.asarray([to_categorical(y_dict[extract_class_from_filename(filename)], len(class_list)) 
                       for filename in filenames])

def make_xy_data(filenames, y_dict):
    x_data = make_x_data(filenames)
    y_data = make_y_data(filenames, y_dict)
    return x_data, y_data

In [ ]:
print('Make train data.......')
with open(train_txt, 'r') as f:
    train_filename_list = [data_dir+line for line in f.read().splitlines()]
x_train, y_train = make_xy_data(train_filename_list, class_dict)
x_train, y_train = shuffle(x_train, y_train)

print('Make validation data.......')
with open(val_txt, 'r') as f:
    val_filename_list = [data_dir+line for line in f.read().splitlines()]
x_val, y_val = make_xy_data(val_filename_list, class_dict)
x_val, y_val = shuffle(x_train, y_train)

print(x_train.shape, y_train.shape, x_val.shape, y_val.shape)

Make train data.......


(3) Create a sequential model

In [ ]:
model=Sequential()

# Layer 1
model.add(Conv1D (kernel_size=3, filters=128, strides=3, padding='valid',
                  kernel_initializer='he_uniform', input_shape=x_train.shape[1:]))                  
model.add(BatchNormalization())
model.add(Activation('relu'))

# Layer 2
model.add(Conv1D (kernel_size=3, filters=128, padding='same', kernel_initializer='he_uniform'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling1D(pool_size=3, strides=3))

# Layer 3
model.add(Conv1D (kernel_size=3, filters=128, padding='same', kernel_initializer='he_uniform'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling1D(pool_size=3, strides=3))

# Layer 4
model.add(Conv1D (kernel_size=3, filters=256, padding='same', kernel_initializer='he_uniform'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling1D(pool_size=3, strides=3))

# Layer 5
model.add(Conv1D (kernel_size=3, filters=256, padding='same', kernel_initializer='he_uniform'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling1D(pool_size=3, strides=3))

# Layer 6
model.add(Conv1D (kernel_size=3, filters=256, padding='same', kernel_initializer='he_uniform'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling1D(pool_size=3, strides=3))

# Layer 7
model.add(Conv1D (kernel_size=3, filters=256, padding='same', kernel_initializer='he_uniform'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling1D(pool_size=3, strides=3))

# # Layer 8
# model.add(Conv1D (kernel_size=3, filters=256, padding='same', kernel_initializer='he_uniform'))
# model.add(BatchNormalization())
# model.add(Activation('relu'))
# model.add(MaxPooling1D(pool_size=3, strides=3))

# # Layer 9
# model.add(Conv1D (kernel_size=3, filters=256, padding='same', kernel_initializer='he_uniform'))
# model.add(BatchNormalization())
# model.add(Activation('relu'))
# model.add(MaxPooling1D(pool_size=3, strides=3))

# # Layer 10
# model.add(Conv1D (kernel_size=3, filters=512, padding='same', kernel_initializer='he_uniform'))
# model.add(BatchNormalization())
# model.add(Activation('relu'))
# model.add(MaxPooling1D(pool_size=3, strides=3))

# Layer 11
model.add(Dropout(0.5))
model.add(Flatten())

# Layer 12
model.add(Dense(16))
model.add(Activation('softmax'))
model.summary()

(4) Compile 

In [ ]:
model.compile(loss='categorical_crossentropy', 
              optimizer=keras.optimizers.SGD(lr=0.01, momentum=0.9, decay=1e-6, nesterov=True),
              metrics=['accuracy'])

(5) Train

In [18]:
model_path = '/users/lww/data/checkpoint/SampleCNN_based_7conv_check_point/'
os.makedirs(model_path, exist_ok=True)
model_filename = model_path+'{epoch:02d}-{val_loss:.4f}.hdf5'
checkpointer = ModelCheckpoint(filepath = model_filename, monitor = "val_loss", verbose=1, save_best_only=True)

early_stopping = EarlyStopping(monitor='val_loss', patience=50)

In [19]:
hist = model.fit(x_train, y_train, batch_size=16, epochs=10000, validation_data=[x_val, y_val], 
                 shuffle=True, callbacks = [checkpointer, early_stopping])

Train on 40951 samples, validate on 40951 samples
Epoch 1/10000
40951/40951 [==============================] - 287s 7ms/step - loss: 1.5100 - acc: 0.5650 - val_loss: 0.6894 - val_acc: 0.7771

Epoch 00001: val_loss improved from inf to 0.68941, saving model to /users/lww/data/checkpoint/SampleCNN_based_check_point/01-0.6894.hdf5
Epoch 2/10000
40951/40951 [==============================] - 286s 7ms/step - loss: 0.5388 - acc: 0.8307 - val_loss: 0.3632 - val_acc: 0.8886

Epoch 00002: val_loss improved from 0.68941 to 0.36317, saving model to /users/lww/data/checkpoint/SampleCNN_based_check_point/02-0.3632.hdf5
Epoch 3/10000
40951/40951 [==============================] - 286s 7ms/step - loss: 0.3962 - acc: 0.8754 - val_loss: 0.2640 - val_acc: 0.9179

Epoch 00003: val_loss improved from 0.36317 to 0.26398, saving model to /users/lww/data/checkpoint/SampleCNN_based_check_point/03-0.2640.hdf5
Epoch 4/10000
40951/40951 [==============================] - 286s 7ms/step - loss: 0.3224 - acc: 0.898


Epoch 00032: val_loss did not improve from 0.03055
Epoch 33/10000
40951/40951 [==============================] - 285s 7ms/step - loss: 0.0582 - acc: 0.9812 - val_loss: 0.0285 - val_acc: 0.9931

Epoch 00033: val_loss improved from 0.03055 to 0.02848, saving model to /users/lww/data/checkpoint/SampleCNN_based_check_point/33-0.0285.hdf5
Epoch 34/10000
40951/40951 [==============================] - 285s 7ms/step - loss: 0.0569 - acc: 0.9815 - val_loss: 0.0323 - val_acc: 0.9907

Epoch 00034: val_loss did not improve from 0.02848
Epoch 35/10000
40951/40951 [==============================] - 284s 7ms/step - loss: 0.0487 - acc: 0.9840 - val_loss: 0.0228 - val_acc: 0.9944

Epoch 00035: val_loss improved from 0.02848 to 0.02283, saving model to /users/lww/data/checkpoint/SampleCNN_based_check_point/35-0.0228.hdf5
Epoch 36/10000
40951/40951 [==============================] - 284s 7ms/step - loss: 0.0471 - acc: 0.9852 - val_loss: 0.0254 - val_acc: 0.9926

Epoch 00036: val_loss did not improve fro

40951/40951 [==============================] - 284s 7ms/step - loss: 0.0101 - acc: 0.9966 - val_loss: 0.0069 - val_acc: 0.9982

Epoch 00106: val_loss did not improve from 0.00180
Epoch 107/10000
40951/40951 [==============================] - 285s 7ms/step - loss: 0.0119 - acc: 0.9965 - val_loss: 0.0020 - val_acc: 0.9997

Epoch 00107: val_loss did not improve from 0.00180
Epoch 108/10000
40951/40951 [==============================] - 285s 7ms/step - loss: 0.0065 - acc: 0.9979 - val_loss: 0.0015 - val_acc: 0.9998

Epoch 00108: val_loss improved from 0.00180 to 0.00147, saving model to /users/lww/data/checkpoint/SampleCNN_based_check_point/108-0.0015.hdf5
Epoch 109/10000
40951/40951 [==============================] - 285s 7ms/step - loss: 0.0076 - acc: 0.9978 - val_loss: 0.0023 - val_acc: 0.9996

Epoch 00109: val_loss did not improve from 0.00147
Epoch 110/10000
40951/40951 [==============================] - 284s 7ms/step - loss: 0.0074 - acc: 0.9978 - val_loss: 0.0016 - val_acc: 0.9998



40951/40951 [==============================] - 281s 7ms/step - loss: 0.0040 - acc: 0.9989 - val_loss: 5.7297e-04 - val_acc: 0.9999

Epoch 00181: val_loss did not improve from 0.00039
Epoch 182/10000
40951/40951 [==============================] - 281s 7ms/step - loss: 0.0035 - acc: 0.9990 - val_loss: 4.0882e-04 - val_acc: 0.9999

Epoch 00182: val_loss did not improve from 0.00039
Epoch 183/10000
40951/40951 [==============================] - 281s 7ms/step - loss: 0.0040 - acc: 0.9987 - val_loss: 5.6315e-04 - val_acc: 0.9998

Epoch 00183: val_loss did not improve from 0.00039
Epoch 184/10000
40951/40951 [==============================] - 281s 7ms/step - loss: 0.0023 - acc: 0.9993 - val_loss: 3.6883e-04 - val_acc: 0.9999

Epoch 00184: val_loss improved from 0.00039 to 0.00037, saving model to /users/lww/data/checkpoint/SampleCNN_based_check_point/184-0.0004.hdf5
Epoch 185/10000
40951/40951 [==============================] - 281s 7ms/step - loss: 0.0033 - acc: 0.9990 - val_loss: 3.6131e-04


Epoch 00218: val_loss did not improve from 0.00026
Epoch 219/10000
40951/40951 [==============================] - 281s 7ms/step - loss: 0.0022 - acc: 0.9994 - val_loss: 2.8948e-04 - val_acc: 0.9999

Epoch 00219: val_loss did not improve from 0.00026
Epoch 220/10000
40951/40951 [==============================] - 283s 7ms/step - loss: 0.0030 - acc: 0.9989 - val_loss: 3.2025e-04 - val_acc: 0.9999

Epoch 00220: val_loss did not improve from 0.00026
Epoch 221/10000
40951/40951 [==============================] - 281s 7ms/step - loss: 0.0034 - acc: 0.9989 - val_loss: 3.3079e-04 - val_acc: 0.9999

Epoch 00221: val_loss did not improve from 0.00026
Epoch 222/10000
40951/40951 [==============================] - 282s 7ms/step - loss: 0.0020 - acc: 0.9995 - val_loss: 3.3547e-04 - val_acc: 0.9999

Epoch 00222: val_loss did not improve from 0.00026
Epoch 223/10000
40951/40951 [==============================] - 281s 7ms/step - loss: 0.0019 - acc: 0.9994 - val_loss: 5.2271e-04 - val_acc: 0.9999

Epoc

...


Epoch 00846: val_loss did not improve from 0.00013
Epoch 847/10000
40951/40951 [==============================] - 284s 7ms/step - loss: 2.3956e-04 - acc: 0.9999 - val_loss: 1.4894e-04 - val_acc: 0.9999

Epoch 00847: val_loss did not improve from 0.00013
Epoch 848/10000
40951/40951 [==============================] - 284s 7ms/step - loss: 2.0253e-04 - acc: 0.9999 - val_loss: 1.5167e-04 - val_acc: 0.9999

Epoch 00848: val_loss did not improve from 0.00013
Epoch 849/10000
40951/40951 [==============================] - 284s 7ms/step - loss: 3.7541e-04 - acc: 0.9999 - val_loss: 1.5250e-04 - val_acc: 0.9999

Epoch 00849: val_loss did not improve from 0.00013
Epoch 850/10000
40951/40951 [==============================] - 284s 7ms/step - loss: 2.4387e-04 - acc: 0.9999 - val_loss: 1.4921e-04 - val_acc: 0.9999

Epoch 00850: val_loss did not improve from 0.00013
Epoch 851/10000
40951/40951 [==============================] - 284s 7ms/step - loss: 3.3325e-04 - acc: 0.9999 - val_loss: 1.5014e-04 - v

40951/40951 [==============================] - 284s 7ms/step - loss: 1.9111e-04 - acc: 0.9999 - val_loss: 1.3876e-04 - val_acc: 1.0000

Epoch 00926: val_loss did not improve from 0.00013
Epoch 927/10000
40951/40951 [==============================] - 284s 7ms/step - loss: 2.8666e-04 - acc: 0.9999 - val_loss: 1.3977e-04 - val_acc: 1.0000

Epoch 00927: val_loss did not improve from 0.00013
Epoch 928/10000
40951/40951 [==============================] - 284s 7ms/step - loss: 1.9080e-04 - acc: 0.9999 - val_loss: 1.4038e-04 - val_acc: 1.0000

Epoch 00928: val_loss did not improve from 0.00013
Epoch 929/10000
40951/40951 [==============================] - 284s 7ms/step - loss: 3.3740e-04 - acc: 0.9999 - val_loss: 1.4067e-04 - val_acc: 1.0000

Epoch 00929: val_loss did not improve from 0.00013
Epoch 930/10000
40951/40951 [==============================] - 284s 7ms/step - loss: 2.6462e-04 - acc: 0.9999 - val_loss: 1.3792e-04 - val_acc: 1.0000

Epoch 00930: val_loss did not improve from 0.00013
E

40951/40951 [==============================] - 284s 7ms/step - loss: 1.7968e-04 - acc: 0.9999 - val_loss: 1.4029e-04 - val_acc: 0.9999

Epoch 00965: val_loss did not improve from 0.00013
Epoch 966/10000
40951/40951 [==============================] - 284s 7ms/step - loss: 2.3900e-04 - acc: 0.9999 - val_loss: 1.3997e-04 - val_acc: 0.9999

Epoch 00966: val_loss did not improve from 0.00013
Epoch 967/10000
40951/40951 [==============================] - 284s 7ms/step - loss: 1.5522e-04 - acc: 1.0000 - val_loss: 1.3989e-04 - val_acc: 0.9999

Epoch 00967: val_loss did not improve from 0.00013
Epoch 968/10000
40951/40951 [==============================] - 285s 7ms/step - loss: 1.6556e-04 - acc: 1.0000 - val_loss: 1.4151e-04 - val_acc: 0.9999

Epoch 00968: val_loss did not improve from 0.00013
Epoch 969/10000
40951/40951 [==============================] - 284s 7ms/step - loss: 1.6210e-04 - acc: 0.9999 - val_loss: 1.3798e-04 - val_acc: 0.9999

Epoch 00969: val_loss did not improve from 0.00013
E

40951/40951 [==============================] - 284s 7ms/step - loss: 1.8133e-04 - acc: 1.0000 - val_loss: 1.5118e-04 - val_acc: 0.9999

Epoch 01044: val_loss did not improve from 0.00013
Epoch 1045/10000
40951/40951 [==============================] - 284s 7ms/step - loss: 1.9672e-04 - acc: 0.9999 - val_loss: 1.4447e-04 - val_acc: 0.9999

Epoch 01045: val_loss did not improve from 0.00013
Epoch 1046/10000
40951/40951 [==============================] - 284s 7ms/step - loss: 2.1606e-04 - acc: 1.0000 - val_loss: 1.5508e-04 - val_acc: 0.9999

Epoch 01046: val_loss did not improve from 0.00013
Epoch 1047/10000
40951/40951 [==============================] - 284s 7ms/step - loss: 1.7796e-04 - acc: 1.0000 - val_loss: 1.5275e-04 - val_acc: 0.9999

Epoch 01047: val_loss did not improve from 0.00013
Epoch 1048/10000
40951/40951 [==============================] - 284s 7ms/step - loss: 2.7751e-04 - acc: 0.9999 - val_loss: 1.4564e-04 - val_acc: 0.9999

Epoch 01048: val_loss did not improve from 0.000

KeyboardInterrupt: 

In [ ]:
fig, loss_ax = plt.subplots()
loss_ax.plot(hist.history['loss'], 'y', label='train loss')
loss_ax.plot(hist.history['val_loss'], 'r', label='val loss')
loss_ax.set_xlabel('epoch')
loss_ax.set_ylabel('loss')
loss_ax.legend(loc='upper left')

plt.show()

(6) Test

In [20]:
print('Make test data.......')
with open(test_txt, 'r') as f:
    test_filename_list = [data_dir+line for line in f.read().splitlines()]
x_test, y_test = make_xy_data(test_filename_list, class_dict)
x_test, y_test = shuffle(x_test, y_test)

Make test data.......


In [21]:
model = load_model(model_path+'980-0.0001.hdf5')

In [22]:
[loss, accuracy] = model.evaluate(x_test, y_test)
print('Loss:', loss, 'Accuracy:', accuracy)

5297/5297 [==============================] - 10s 2ms/step
Loss: 0.30202533950785987 Accuracy: 0.956201623583011


In [ ]:
pred = model.predict(x_test)
test_f1_score = f1_score(y_test, pred > 0.5)
print('F1 Score:', test_f1_score)

In [ ]:
%timeit -n 5 -r 5 model.predict(x_test)